In [1]:
import os

In [2]:
%pwd

'd:\\Project\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Project\\Kidney-Disease-Classification-Deep-Learning-Project'

In [ ]:
import os
from pathlib import Path
import tensorflow as tf
from dataclasses import dataclass
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/kidney-ct-scan-image"),
            #UPDATED TO YOUR REPO
            mlflow_uri="https://dagshub.com/aryan18s/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

# ---------------------------------------------------------
# 3. EVALUATION CLASS
# ---------------------------------------------------------
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()

        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        
        dagshub.init(
            repo_owner='aryan18s', 
            repo_name='Kidney-Disease-Classification-Deep-Learning-Project', 
            mlflow=True
        )
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        # --- NEW FIX: Clear memory and force a new experiment ---
        if mlflow.active_run():
            mlflow.end_run()
            
        mlflow.set_experiment("Kidney_Project_V1") 
        # --------------------------------------------------------
        
        with mlflow.start_run():
            
            # Convert lists to text so MLflow doesn't crash
            clean_params = {}
            for key, value in self.config.all_params.items():
                clean_params[key] = str(value)
                
            mlflow.log_params(clean_params)
            
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
    print("✅ Evaluation and MLflow logging completed successfully!")
except Exception as e:
    raise e

[2026-02-24 11:33:08,056: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-24 11:33:08,060: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-24 11:33:08,062: INFO: common: created directory at: artifacts]
[2026-02-24 11:33:08,280: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9281 - loss: 0.1955
[2026-02-24 11:33:20,778: INFO: common: json file saved at: scores.json]
[2026-02-24 11:33:22,004: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/aryan18s/Kidney-Disease-Classification-Deep-Learning-Project "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "aryan18s/Kidney-Disease-Classification-Deep-Learning-Project"

[2026-02-24 11:33:22,009: INFO: helpers: Initialized MLflow to track repo "aryan18s/Kidney-Disease-Classification-Deep-Learning-Project"]


Repository aryan18s/Kidney-Disease-Classification-Deep-Learning-Project initialized!

[2026-02-24 11:33:22,012: INFO: helpers: Repository aryan18s/Kidney-Disease-Classification-Deep-Learning-Project initialized!]


2026/02/24 11:33:22 INFO mlflow.tracking.fluent: Experiment with name 'Kidney_Project_V1' does not exist. Creating a new experiment.
2026/02/24 11:33:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/24 11:33:27 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


[2026-02-24 11:33:53,361: WARNING: connectionpool: Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /aryan18s/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/api/2.0/mlflow/logged-models/m-d90770ada6c34e96bac3c263f322d1ee]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2026/02/24 11:34:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.


🏃 View run amusing-donkey-902 at: https://dagshub.com/aryan18s/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/1/runs/45ecf50f2ae842718961f8989d1fd0f9
🧪 View experiment at: https://dagshub.com/aryan18s/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/1
✅ Evaluation and MLflow logging completed successfully!
